In [5]:
### This notebook is used to collect all season results available on College Hockey News

# There Records begin with the 1901-1902 season and end with the most recent completed season 2022-2023
## Seasons begining in 2002-2003 have box score links in the results table - seasons from then on can 
## use the code develope4d in the other notbook, years before that will need some adjusted code

## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


## Example URL: https://www.collegehockeynews.com/schedules/?season=19851986

base_url = 'https://www.collegehockeynews.com/schedules/?season='

# Construct the url for each season and store in a list
mod_seasons = []

for i in range(2002, 2023):
    mod_seasons.append(base_url + str(i) + str(i+1))

print(len(mod_seasons))

old_seasons = []

for i in range(1901, 2002):
    old_seasons.append(base_url + str(i) + str(i+1))

print(len(old_seasons))



21
101


In [6]:
### Scraping code from initial scrape_and_explore notebook

############### Test with one season ######################
url = 'https://www.collegehockeynews.com/schedules/?season=20012002'


def parse_modern_season(url):
    
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    # data = []

    # Parse the table with BeautifulSoup

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
            

## Try running the function on the list of urls and create dataframe after looping through all seasons

# Initialize an empty list to hold the data
data = []

for url in mod_seasons:
    parse_modern_season(url)
    # wait 2 seconds between requests
    time.sleep(2)


# Create a DataFrame
columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)


# ## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# # remove day of the week from date
# # format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
## Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)




C:\Users\Justin\AppData\Local\Temp\ipykernel_10992\3339462295.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2002-10-01_Windsor_Green' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row.Index, 'Game_ID'] = game_id


In [7]:
print(len(df))

df.head(20)

# Save the dataframe to a csv file
df.to_csv('../data/results_table_2002_2022_new.csv', index=False)

24022


In [8]:
### Code for seasons 2001-2002 and earlier

example_url = 'https://www.collegehockeynews.com/schedules/?season=19811982'

## list of old seasons is stored in old_seasons


# Update the function to include game notes and overtime information
def parse_pre_2002_season(url):
    # Get the page with requests
    response = requests.get(url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Initialize variables
    current_date = None
    current_conference = None

    # Initialize an empty list to hold the data
    # data = []

    # Parse the table with BeautifulSoup
    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 5:
                home_team = cells[0].text.strip()
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                
                # Extract overtime information
                ot = cells[5].text.strip() if cells[5].text.strip() else None
                
                # Extract game notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, game_notes])

    


## Loop through the list of urls and run the function on each one
data = []

for url in old_seasons:
    parse_pre_2002_season(url)
    # wait 2 seconds between requests
    time.sleep(2)


# create a dataframe from the data
columns = ['Date', 'Conference', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Game_Notes']
df = pd.DataFrame(data, columns=columns)

## Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)


print(len(df))
df.head(20)


In [ ]:
# ## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# # remove day of the week from date
# # format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [ ]:
## Save the dataframe to a csv file
df.to_csv('../data/results_table_1901_2001_new.csv', index=False)